In [1]:
!pip install transformers

Looking in indexes: https://pypi.org/simple, https://pypi.ngc.nvidia.com


In [2]:
!pip install tensorflow-gpu

Looking in indexes: https://pypi.org/simple, https://pypi.ngc.nvidia.com


In [3]:
from tensorflow.python.client import device_lib

def get_available_devices():
    local_device_protos = device_lib.list_local_devices()
    return [x.name for x in local_device_protos]

print(get_available_devices())

2022-10-31 14:19:18.336674: I tensorflow/core/platform/cpu_feature_guard.cc:193] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2 FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.
2022-10-31 14:19:18.502936: E tensorflow/stream_executor/cuda/cuda_blas.cc:2981] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
2022-10-31 14:19:19.099549: W tensorflow/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libnvinfer.so.7'; dlerror: libnvinfer.so.7: cannot open shared object file: No such file or directory; LD_LIBRARY_PATH: /usr/local/cuda/compat/lib.real:/opt/conda/lib/python3.8/site-packages/torch/lib:/opt/conda/lib/python3.8/site-packages/torch_tensorrt/lib:/usr/local/cuda/compat/lib:/usr/local/nvidia/lib:/usr/local/nvidia/lib64


['/device:CPU:0', '/device:GPU:0', '/device:GPU:1', '/device:GPU:2', '/device:GPU:3', '/device:GPU:4', '/device:GPU:5', '/device:GPU:6', '/device:GPU:7']


2022-10-31 14:19:20.009593: I tensorflow/core/platform/cpu_feature_guard.cc:193] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2 FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.
2022-10-31 14:19:23.985581: I tensorflow/core/common_runtime/gpu/gpu_device.cc:1616] Created device /device:GPU:0 with 35440 MB memory:  -> device: 0, name: A100-SXM4-40GB, pci bus id: 0000:07:00.0, compute capability: 8.0
2022-10-31 14:19:23.987570: I tensorflow/core/common_runtime/gpu/gpu_device.cc:1616] Created device /device:GPU:1 with 35440 MB memory:  -> device: 1, name: A100-SXM4-40GB, pci bus id: 0000:0f:00.0, compute capability: 8.0
2022-10-31 14:19:23.989268: I tensorflow/core/common_runtime/gpu/gpu_device.cc:1616] Created device /device:GPU:2 with 35440 MB memory:  -> device: 2, name: A100-SXM4-40GB, pci bus id: 0000:47:00.0, compute capab

In [4]:
!pip install sentencepiece

Looking in indexes: https://pypi.org/simple, https://pypi.ngc.nvidia.com


In [5]:
!pip install seaborn

Looking in indexes: https://pypi.org/simple, https://pypi.ngc.nvidia.com


In [6]:
!pip install tqdm

Looking in indexes: https://pypi.org/simple, https://pypi.ngc.nvidia.com


In [7]:
import pandas as pd
import numpy as np
from transformers import pipeline
from tqdm import tqdm
from transformers import PegasusForConditionalGeneration, PegasusTokenizer
import time
from transformers import AdamW, AutoTokenizer, AutoModelForSequenceClassification
from transformers import DataCollatorWithPadding
from transformers import AutoTokenizer, AutoModelForSequenceClassification, TrainingArguments, Trainer

In [8]:
!pip install ipywidgets

Looking in indexes: https://pypi.org/simple, https://pypi.ngc.nvidia.com


In [9]:
!jupyter nbextension enable --py widgetsnbextension

Enabling notebook extension jupyter-js-widgets/extension...
      - Validating: OK


In [10]:
import ast

In [11]:
%cd ..

/workspace/asurion_f22_muby


In [12]:
transcript = pd.read_csv("fe_data_v2/transcript_data_use/mock_.csv") # replace mock_transcript.csv with transcript_chunked.csv
transcript["encrypted_collated_transcription"] = transcript["encrypted_collated_transcription"].astype(str)

In [13]:
transcript.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1000 entries, 0 to 999
Data columns (total 6 columns):
 #   Column                            Non-Null Count  Dtype 
---  ------                            --------------  ----- 
 0   id                                1000 non-null   object
 1   sales_offer_date                  1000 non-null   object
 2   accepted_flg                      1000 non-null   bool  
 3   encrypted_collated_transcription  1000 non-null   object
 4   month                             1000 non-null   object
 5   chunked                           1000 non-null   object
dtypes: bool(1), object(5)
memory usage: 40.2+ KB


In [14]:
list_of_sentence = transcript["chunked"].map(ast.literal_eval)
list_of_sentence[0]

["ok Thank for calling [ CLIENT ] Tech Hello? Hi. It's Christie Ferguson. Speaking. May I have your first and last name, please? Hi there, Christie. What's the phone number calling about? I'm calling about mobile, um, data use constantly anymore. Ok. Well, sorry here they have the issues with regards your Mobile data about, Christie. What's the phone number you're calling about? Seven [ NUMBER ] [ NUMBER ] [ NUMBER ] [ NUMBER ] [ NUMBER ] [ NUMBER ] [ NUMBER ] [ NUMBER ] seven Thank you so much. Ok. Let me just pull up your account here, Chris. so Could be able to run a network test and then uh later on let's make sure that all of the security pictures activated onto your account. So if could be able to maximize the uh, functionality see every phone. About the way Christie, uh, you mentioned and that you're having troubles when it comes to your cellular data Can you tell me since when it started happening? been going on and off for probably a couple weeks, but if it's not Probably. kno

In [15]:
transcript["chunked"] = transcript["chunked"].map(ast.literal_eval)
transcript["chunked"][0]

["ok Thank for calling [ CLIENT ] Tech Hello? Hi. It's Christie Ferguson. Speaking. May I have your first and last name, please? Hi there, Christie. What's the phone number calling about? I'm calling about mobile, um, data use constantly anymore. Ok. Well, sorry here they have the issues with regards your Mobile data about, Christie. What's the phone number you're calling about? Seven [ NUMBER ] [ NUMBER ] [ NUMBER ] [ NUMBER ] [ NUMBER ] [ NUMBER ] [ NUMBER ] [ NUMBER ] seven Thank you so much. Ok. Let me just pull up your account here, Chris. so Could be able to run a network test and then uh later on let's make sure that all of the security pictures activated onto your account. So if could be able to maximize the uh, functionality see every phone. About the way Christie, uh, you mentioned and that you're having troubles when it comes to your cellular data Can you tell me since when it started happening? been going on and off for probably a couple weeks, but if it's not Probably. kno

In [16]:
for i in tqdm(range(len(transcript))):
    transcript.loc[i,"chunk_length"] = len(transcript.loc[i,"chunked"])
transcript.head()

100%|█████████████████████████████████████████████████████████████████████████████| 1000/1000 [00:00<00:00, 5508.50it/s]


,id,sales_offer_date,accepted_flg,encrypted_collated_transcription,month,chunked,chunk_length
0,e5e4ee3c3c0e9ac72561334e6eb23cc233b1bcd0a0063d...,2022-03-26,False,ok Thank for calling [CLIENT] Tech Hello? Hi. ...,march,[ok Thank for calling [ CLIENT ] Tech Hello? H...,4.0
1,ae6128d8d240d28edbcb7eb27256780cf662a2be1aec8c...,2022-03-02,False,"on on Oh, My name is [NAME]. May I please have...",march,"[on on Oh, My name is [ NAME ]. May I please h...",4.0
2,4899f9acdb70b0246b2d0ac2871dbcea8267e57b060154...,2022-03-12,False,Hello. We for calling [CLIENT] Tech Coach. My ...,march,[Hello. We for calling [ CLIENT ] Tech Coach. ...,2.0
3,0589751e11caac0eac815faf84f1a366d70b86b78324a2...,2022-03-20,False,Hello. Thank you so much for contacting [CLIEN...,march,[Hello. Thank you so much for contacting [ CLI...,4.0
4,6c4790cb02e4f556d73436f79268d5c99de50a9acec40e...,2022-03-15,False,Thank you very much Hello? can I can you hear ...,march,[Thank you very much Hello? can I can you hear...,5.0


In [17]:
transcript["chunk_length"] = transcript["chunk_length"].astype(int)

In [18]:
#pipe = pipeline("summarization", model="facebook/bart-large-cnn", device = 3
#from transformers import AutoModelForSeq2SeqLM
#max_memory_mapping = {0: "8GB", 1: "8GB"}
#model = AutoModelForSeq2SeqLM.from_pretrained("facebook/bart-large-cnn", device_map="auto", load_in_8bit=True, max_memory=max_memory_mapping)

In [19]:
pipe = pipeline("summarization", model="facebook/bart-large-cnn", device = 3, framework = "pt", num_workers = 16)
#pipe = pipeline("summarization", model="t5-base", tokenizer="t5-base", framework="tf", device = 2)

Downloading:   0%|          | 0.00/1.58k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/1.63G [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/899k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/456k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/1.36M [00:00<?, ?B/s]

In [20]:
new_list = []
for i in range(len(list_of_sentence)):
    new_list.extend(list_of_sentence[i])
len(new_list)

5706

In [21]:
#pipe_list = []
#for i in tqdm(range(len(list_of_sentence))):
#    pipe_out = pipe(list_of_sentence[i], batch_size = 256)
#    pipe_list.append(pipe_out)
#len(pipe_list)

In [ ]:
start=time.time()
pipe_list = []
n = len(new_list)
for out in tqdm(pipe(new_list, batch_size = 64), total=n):
    pipe_list.append(out)

In [ ]:
print(round(time.time()-start, 2), 'seconds')

In [ ]:
len(pipe_list)

In [ ]:
length = list(transcript["chunk_length"])

In [ ]:
list_of_summary = []
num_init = 0
num_final = 0
for i in range(len(length)):
    num_init = num_final
    num_final = num_init + int(length[i])
    list_of_summary.append(pipe_list[num_init:num_final])

In [ ]:
#tqdm.pandas()
#pipe_list = transcript["chunked"].progress_apply(lambda x: pipe(x, batch_size = 2048))

In [ ]:
final_list = []
for i in tqdm(range(len(list_of_summary))):
    sentence_list = []
    pipe_num = list_of_summary[i]
    for j in range(len(pipe_num)):
        sentence_list.append(pipe_num[j]["summary_text"])
    final_list.append(sentence_list)

In [ ]:
def concatenate_sentence(chunked_list):
    summary = ""
    for i in range(len(chunked_list)):
        summary += chunked_list[i]
    return summary

In [ ]:
summary_lists = []
for i in tqdm(range(len(final_list))):
    summary_lists.append(concatenate_sentence(final_list[i]))

In [ ]:
transcript["summary"] = summary_lists

In [ ]:
string = transcript["summary"][999]
print(string)

In [ ]:
transcript.to_csv("fe_data_v2/transcript_data_use/mock_transcript_summary.csv", index = False) 
# I used transcript_data_use/ as the directory for storing my files. You can create a new folder with a naming of your choice
# And I used mock_transcript_summary.csv as my file name. You can use a nomenclature suitable for parallelizing like data 1, data 2 ...
# since you will be breaking the data into chunks for different notebooks.